In [7]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import tree

In [8]:
def fetch_diabetes_ct_data_one_hot(oppScrData):    
    ct_data= oppScrData[["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'Otherabnormalglucose', 'Impairedfastingglucose', 
       'TypeIIorunspecifiedtypediabetesmellituswithoutmentionofcomplication,uncontrolled', 
       'Type2diabetesmellituswithoutcomplications']];
    n = ct_data.shape[0]
    preprocessed_ct_data = []
    for i in range(n):
        row = ct_data.iloc[i]
        ignore = False
        for j in row[:]:
          if pd.isna(j) or j == ' ': # There is an empty string somewhere in Liver column
            ignore = True
            break
        if not ignore:
          preprocessed_ct_data.append(row)
    return np.array(preprocessed_ct_data, dtype=np.float32)


In [9]:
def fetch_diabetes_ct_data(oppScrData):    
    ct_data= oppScrData[["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'Type 2 Diabetes DX']];
    n = ct_data.shape[0]
    preprocessed_ct_data = []
    for i in range(n):
        row = ct_data.iloc[i]
        ignore = False
        for j in row[:]:
          if pd.isna(j) or j == ' ': # There is an empty string somewhere in Liver column
            ignore = True
            break
        if not ignore:
          preprocessed_ct_data.append(row)
    return np.array(preprocessed_ct_data, dtype=np.float32)


**ONE HOT**

In [ ]:
df = pd.read_excel(r'sample_data/OppScrData.xlsx')  

In [ ]:
# Fetch rows with the below specified diabetes values and convert it to one-hot representation
p = ['Otherabnormalglucose', 'Impairedfastingglucose', 'TypeIIorunspecifiedtypediabetesmellituswithoutmentionofcomplication,uncontrolled', 'Type2diabetesmellituswithoutcomplications'];
df = df.loc[df["Type 2 Diabetes DX"].isin(p)]
diabetes_one_hot = pd.get_dummies(df['Type 2 Diabetes DX'])
df = pd.concat([df, diabetes_one_hot], axis=1)
df = df.drop('Type 2 Diabetes DX', axis=1)

In [10]:
# Fetch corresponding ct data values
diabetes_ct_data = fetch_diabetes_ct_data(df)

In [11]:
diabetes_ct_data[0]

array([106.       , 315.53076  , 588.89233  , 202.31795  , 113.21282  ,
         1.7870586,  -3.181874 , 168.92395  ,  47.814583 , 431.519    ,
        53.       ,   1.       ], dtype=float32)

In [ ]:
X = diabetes_ct_data[:, :-4]
y = diabetes_ct_data[:, -4:]

In [ ]:
y

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

**NORMAL REPRESENTATION**

In [10]:
df = pd.read_excel(r'sample_data/OppScrData.xlsx')  
p = ['Otherabnormalglucose', 'Impairedfastingglucose', 'TypeIIorunspecifiedtypediabetesmellituswithoutmentionofcomplication,uncontrolled', 'Type2diabetesmellituswithoutcomplications'];
df = df.loc[df["Type 2 Diabetes DX"].isin(p)]
r = {"Otherabnormalglucose":1, "Impairedfastingglucose":2, "Type2diabetesmellituswithoutcomplications":3, "TypeIIorunspecifiedtypediabetesmellituswithoutmentionofcomplication,uncontrolled":4}
df["Type 2 Diabetes DX"].replace(r, inplace=True)
diabetes_ct_data = fetch_diabetes_ct_data(df)

In [11]:
X = diabetes_ct_data[:, :-1]
y = diabetes_ct_data[:, -1]

In [14]:
X[0]

array([106.       , 315.53076  , 588.89233  , 202.31795  , 113.21282  ,
         1.7870586,  -3.181874 , 168.92395  ,  47.814583 , 431.519    ,
        53.       ], dtype=float32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)


**NAIVE BAYES**

In [16]:
gnb = GaussianNB().fit(X_train, y_train)
gnb_predictions = gnb.predict(X_test)
 
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print(accuracy)
 
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print(cm)


0.3745247148288973
[[ 29 150  29  12]
 [ 25 144  13   7]
 [  8  23  10   8]
 [ 15  27  12  14]]


**kNN**

In [58]:
knn_pipe = Pipeline([('mms', StandardScaler()),
                     ('knn', KNeighborsClassifier())])
params = [{'knn__n_neighbors': range(2, 10),
         'knn__weights': ['uniform', 'distance'],
         'knn__leaf_size': [15, 20]}]
gs_knn = GridSearchCV(knn_pipe,
                      param_grid=params,
                      scoring='accuracy',
                      cv=3)
gs_knn.fit(X, y)
print(gs_knn.best_params_)
print(gs_knn.score(X, y))

{'knn__leaf_size': 15, 'knn__n_neighbors': 9, 'knn__weights': 'uniform'}
0.5259153590109368


**SVM**

In [ ]:
SVC_pipe = Pipeline([('mms', StandardScaler()),
                     ('svc', SVC())])

params = {'C':[0.1,1,100,1000],
          'kernel':['rbf','poly','sigmoid','linear'],
          'degree':[1,2,3,4,5,6],
          'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}

gs_SVC = GridSearchCV(SVC(),
                      param_grid=params,
                      scoring='accuracy',
                      cv=3)
gs_SVC.fit(X, y)
print(gs_SVC.best_params_)
print(gs_SVC.score(X, y))

In [19]:
from sklearn.metrics.pairwise import rbf_kernel
s = SVC(kernel='poly')
s.fit(X,y)
print(s.score(X, y))

0.43319067998097954


**DECISION TREE**

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
dtclf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)
dtclf.fit(X_train, y_train)
pred = dtclf.predict(X_test)
accuracy = dtclf.score(X_test, y_test)
print(accuracy)

0.3681710213776722


In [50]:
params = {'criterion':['gini','entropy'],
             'max_depth':range(1, 5),
          }

# dt_pipe = Pipeline([('mms', StandardScaler()),
#                      ('tree', ))])

gs_dt = GridSearchCV(tree.DecisionTreeClassifier(),
                      param_grid=params,                      
                      cv=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
gs_dt.fit(X_train, y_train)
print(gs_dt.best_params_)
print(gs_dt.score(X_test, y_test))

{'criterion': 'gini', 'max_depth': 1}
0.44655581947743467


**RANDOM FOREST**

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
params = {'criterion':['gini','entropy'],
             'max_depth':range(1, 5),
          }

# dt_pipe = Pipeline([('mms', StandardScaler()),
#                      ('tree', ))])

gs_dt = GridSearchCV(RandomForestClassifier(),
                      param_grid=params,                      
                      cv=10)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
gs_dt.fit(X, y)
print(gs_dt.best_params_)
print(gs_dt.best_score_)

{'criterion': 'gini', 'max_depth': 4}
0.43697810877905663


# **MLP**

In [2]:
from sklearn.neural_network import MLPClassifier

In [3]:
clf = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)

In [13]:
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(150, 100, 50), max_iter=300, random_state=1)

In [14]:
print(clf.score(X_test, y_test))

0.344106463878327


In [30]:
params = {'hidden_layer_sizes':[(10,5), (6,4), (5,3)],
          'max_iter':[500],
          'activation':['relu'],
          'solver':['adam', 'lbfgs', 'sgd']}

gs_dt = GridSearchCV(MLPClassifier(),
                     param_grid=params,                      
                     cv=10)

gs_dt.fit(X, y)
print(gs_dt.best_params_)
print(gs_dt.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the n

{'activation': 'relu', 'hidden_layer_sizes': (6, 4), 'max_iter': 500, 'solver': 'lbfgs'}
0.43128863528292916


0.43224102911306705